#Install

SDK: https://github.com/anthropics/anthropic-sdk-python

In [ ]:
!pip install anthropic

import os
import json
import pandas as pd
import time
import re
import csv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 846.4/846.4 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.3 MB/s eta 0:00:00



# This is formatted as code


### Import packages

In [2]:
import pathlib
import textwrap


from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

  # Used to securely store your API key
from google.colab import userdata

### Setup your API key

Before you can use the Cluade API, you must first obtain an API key. If you don't already have one, create a key with one click in Google AI Studio.

<a class="button button-primary" href="https://console.anthropic.com/settings/keys" target="_blank" rel="noopener noreferrer">Get an API key</a>


In Colab, add the key to the secrets manager under the "🔑" in the left panel. Give it the name `CLUADE_API_KEY`.

Once you have the API key, pass it to the SDK. You can do this in two ways:

* Put the key in the `CLUADE_API_KEY` environment variable (the SDK will automatically pick it up from there).


In [4]:
import os
from anthropic import Anthropic

client = Anthropic(
    # This is the default and can be omitted
    # Or use `os.getenv('finalproject-secret-key')` to fetch an environment variable.
    #CLUADE_API_KEY=userdata.get('finalproject-secret-key')
    api_key=userdata.get("ANTHROPIC_API_KEY"),
)


### Demo test for us

In [5]:
message = client.messages.create(
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": "How does a court case get to the supreme court?",
        }
    ],
    model="claude-2.1",
    temperature=0,
)



print(message.content)

BadRequestError: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'Your credit balance is too low to access the Claude API. Please go to Plans & Billing to upgrade or purchase credits.'}}

### Data:
Our data will be taken from 162 snort rules that have already been manually labeled to techniques from MITRE ATT&CK.

In [ ]:
!git clone https://github.com/trickdeath0/Labeling_IDS_to_MITRE.git

In [ ]:
data = pd.read_csv('/content/Labeling_IDS_to_MITRE/ground_truth.csv')
print(data.head())
rules_list = data['Rule']
true_labels = data['technique ids']

#print(data['Sid'][0+41])

In [ ]:
def clean_response(text):
    text = text.data.replace(">", "").strip()  # Remove leading ">", whitespace
    try:
      text = text.replace("```json", "")
      text = text.replace("```", "")
    except:
      pass
    return text


# **Prompting without techniques guide (WTG):**
At this stage, the LLMs will receive a prompt that does not include the list of techniques from MITRE ATT&CK in order to examine the results of the models based on prior knowledge that has been trained. According to our request, the LLMs will classify the techniques according to the content of the rule.


In [ ]:
def wtg(snort_rule):
  # response = model.generate_content(f"""You are an information security expert. Your task is to label IDS rules for MITRE ATT&CK techniques based on your cybersecurity knowledge. For the task, you are going to get a single Snort IDS rule and you will need to return the most relevant techniques from MITRE ATT&CK that are related to the rule.
  # Try to search based on keywords and based on the knowledge you have. For each technique include the following information as JSON in this order: 'Sid', 'Technique ID', 'Technique Name', 'Quotes', 'Explanation'.
  # Note: The value of the citation field should contain quotation marks from the data sets relevant to the mapped technique are the main reason you chose this technique to be correct. The value of the explanation should be your explanation of why you decided to give the technique and how it relates to the rule. The technique ID should be the official MITRE technique ID.
  # Please don't write anything but the JSON. Rule: {snort_rule}""")


  # response = model.generate_content(f"""You work in a company that deals with information security, your role in the company is to label techniques from MITRE ATT&CK to the rules of IDS systems. The labeling between a rule and a technique indicates that the attacker operated with a technique that you found to be suitable for the rule that alerted the IDS system. Now we will test your knowledge labeling IDS rules for MITRE ATT&CK techniques. For your task, you're going to have a single Snort IDS rule and you'll need to label the most relevant techniques from MITRE ATT&CK associated with the rule. From the rule you receive, your labeling should be based on your knowledge and the information found within the 'msg' in the rule received. For each technique you call the rule, include the following information as JSON format in this order: 'Sid', 'Technique ID', 'Technique Name', 'Quotes', 'Explanation'.  Note: The value of the 'Quotes' field should contain quotation marks from the data sets relevant to the mapped technique. The value of the 'Explanation' should be your explanation of why you decided to give the technique and how it relates to the rule. The 'Technique ID' should be the official MITRE technique ID.
  # Please don't write anything but the JSON. Rule: {snort_rule}""")

  response = model.generate_content(f"""
  I'm going to give you a Snort rule. Read the Snort rule carefully, because I'm going to given you a task about it. Here is the Snort rule: <snort_rule>{snort_rule}</snort_rule>

  First, find the techniques from MITRE ATT&CK that are most relevant to the Snort rule.

  Then, answer the task, for each technique include the following information as JSON in this order: 'Sid', 'Technique ID', 'Technique Name', 'Quotes', 'Explanation'.

  Note: The value of the citation field should contain quotation marks from the data sets relevant to the mapped technique are the main reason you chose this technique to be correct. The value of the explanation should be your explanation of why you decided to give the technique and how it relates to the rule. The technique ID should be the official MITRE technique ID.

  Thus, the format of your overall response should look like what's shown between the <examples></examples> tags. Make sure to follow the formatting and spacing exactly.


  <examples>
  [
    "sid": "2274",
    "Technique ID": "T1110",
    "Technique name": "Brute Force",
    "Quotes": [
      "\"PROTOCOL-POP login brute force attempt\"",
      "track by_dst,count 30,seconds 30"
    ],
    "Explanation": "The rule is looking for excessive \"USER\" commands within a short period of time, which are common indicators of brute-force attacks targeting the POP3 service."
  ]
  </examples>

  Do not include anything besides write the JSON.
  """)

  # response = model.generate_content(f'''You are going to receive a Snort rule and your task is to find as many MITRE ATT&CK techniques as possible that are associated with the rule. Note: You should categorize the techniques to 1 or 2. Technique of type 1 is a technique that you can associate with the rule directly based on the rule. Technique of type 2 is a technique that can be associated with the rule indirectly, based on your knowledge and understanding. The categorization value should be the value 1 or 2, based on the explanation given above. The quotes field value should contain quotes from the rules data that are relevant to the technique mapped and they are the main reason you believe the mapping to this technique is correct. The explanation’s value should be your explanation for why you decided to give the technique and how it is associated with the rule. The technique id should be the official MITRE technique id. For each technique include the following information as JSON: sid, Technique id, Technique name, Categorization, Quotes, Explanation. After each rule I will provide you with, answer according to the provided format. Please do not write anything else but the JSON. Rule: {snort_rule}''')

  return response


In [ ]:
rule_dict = {}
max_retries = 3  # Maximum number of retries

for index, rule in enumerate(rules_list):
    retries = 0
    while retries < max_retries:
        try:
            res = wtg(rule)
            text = res.text
            # Check if the text contains the desired pattern
            t_numbers = re.findall(r'[\'\"](T\d+(?:\.\d+)?)', text)
            if t_numbers:  # If the pattern is found
                rule_dict[data['Sid'][index]] = to_markdown(text)
                break  # Break out of the retry loop if successful
            else:
                print("Desired pattern not found in the text. Retrying...")
                retries += 1
                time.sleep(1)  # Wait for a short duration before retrying
        except Exception as e:
            print(f"An error occurred: {e}")
            retries += 1
            if retries < max_retries:
                print(f"Retrying... ({retries}/{max_retries})")
                time.sleep(1)  # Wait for a short duration before retrying
            else:
                print("Max retries reached. Unable to process this rule.")

# If sending fails, attempt to send again
try:
    # Code to send data
    pass
except Exception as e:
    print(f"Sending failed: {e}")
    # Retry sending here
